In [114]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib
import matplotlib.pyplot as plt
import xgboost as xgb

from sklearn.linear_model import Ridge, Lasso, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.cross_validation import cross_val_score
from scipy.stats import skew

''' Helper function for calculating RMSE '''
def rmse_cv(model):
    rmse= np.sqrt(-cross_val_score(model, X_train, y, scoring="neg_mean_squared_error", cv = 5))
    return(rmse)

''' Pre-processing '''
train = pd.read_csv('/Users/anthonydepalatis/Documents/School/EE379K/Lab4/train.csv')
test = pd.read_csv('/Users/anthonydepalatis/Documents/School/EE379K/Lab4/test.csv')
all_data = pd.concat((train.loc[:,'MSSubClass':'SaleCondition'],
                      test.loc[:,'MSSubClass':'SaleCondition']))

# Log transform the classification set
train["SalePrice"] = np.log1p(train["SalePrice"])
prices = pd.DataFrame(train["SalePrice"])

# Log transform skewed numeric features
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index
skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index
all_data[skewed_feats] = np.log1p(all_data[skewed_feats])

# Convert categorical features into dummy variables
all_data = pd.get_dummies(all_data)

# Fill empty cells with the mean of the column
all_data = all_data.fillna(all_data.mean())

# Create matrices for sklearn
X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]
y = train.SalePrice

''' Part 2 '''
# Train a ridge regression model
my_alphas = [0.01, 0.05, 0.1, 0.5, 1, 2, 4, 8, 10, 15, 30, 45, 60]
model_ridge = [rmse_cv(Ridge(alpha=alpha, solver='svd', tol=0.001)).mean() for alpha in my_alphas]
model_ridge = pd.Series(model_ridge, index = alphas)
print 'Ridge Regression Model Error: ' + str(model_ridge.min())

# Train a lasso regression model
my_alphas = [0.1, 0.01, 0.005, 0.001, 0.0005, 0.0001, 0.00005]
model_lasso = LassoCV(alphas=my_alphas).fit(X_train, y)
print 'Lasso Regression Model Error: ' + str(rmse_cv(model_lasso).mean())

''' Part 3 '''
# Get the total nonzero coefficients output by the lasso model for each alpha value
coefs = []
for alpha in my_alphas:
    model_lasso = Lasso(alpha=alpha).fit(X_train, y)
    coef = pd.Series(model_lasso.coef_, index=X_train.columns)
    coefs.append(sum(coef == 0))
    
# Plot the number of nonzero coefficients vs. alpha
l_0 = pd.Series(coefs, index = my_alphas)
l_0.plot(title="l0 vs. Alpha")
plt.xlabel("alpha")
plt.ylabel("# nonzero coefs")
plt.show()

''' Part 4 '''

OSError: dlopen(/Users/anthonydepalatis/anaconda/lib/python2.7/site-packages/libxgboostwrapper.so, 6): Library not loaded: @rpath/./libgomp.1.dylib
  Referenced from: /Users/anthonydepalatis/anaconda/lib/python2.7/site-packages/libxgboostwrapper.so
  Reason: image not found